In [11]:
import os
import re
import PyPDF2
import json
import torch
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from dotenv import load_dotenv

In [3]:
from huggingface_hub import login

login(os.getenv('TOKEN'))

In [98]:
model_name = "mental/mental-bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModel.from_pretrained(model_name, use_auth_token=True)

c:\Pyhton\Python310\lib\site-packages\transformers\models\auto\tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Pyhton\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

c:\Pyhton\Python310\lib\site-packages\transformers\models\auto\auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

c:\Pyhton\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

c:\Pyhton\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [172]:
from transformers import AutoModelForCausalLM

tokenizer2 = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model2 = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

c:\Pyhton\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
whitespace_pattern = re.compile(r'\s+')

# Pattern to remove special characters and punctuation (except apostrophes)
special_char_pattern = re.compile(r'[^\w\s\']')

# Pattern to fix inconsistent apostrophe usage
apostrophe_pattern = re.compile(r'\s\'|\'\s')

# Pattern to remove URLs
url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

# Pattern to remove email addresses
email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

# Pattern to remove numbers followed by specific words (e.g., "7/29/24, 5:03 pm")
date_time_pattern = re.compile(r'\d+/\d+/\d+,?\s+\d+:\d+\s+[ap]m')

# Pattern to remove text within parentheses
parentheses_pattern = re.compile(r'\([^()]*\)')

In [5]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def clean_text(text):
    text = text.lower()

    text = url_pattern.sub('', text)
    
    # Remove email addresses
    text = email_pattern.sub('', text)
    
    # Remove date and time patterns
    text = date_time_pattern.sub('', text)
    
    # Remove text within parentheses
    text = parentheses_pattern.sub('', text)
    
    # Remove special characters (except apostrophes)
    text = special_char_pattern.sub(' ', text)
    
    # Fix inconsistent apostrophe usage
    text = apostrophe_pattern.sub("'", text)
    
    # Remove extra whitespace and newlines
    text = whitespace_pattern.sub(' ', text)
    
    text = text.strip()

    text = ' '.join(text.split())
    return text

def create_embedding(text, model, tokenizer):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

In [91]:
# t = extract_text_from_pdf('1_blog.pdf')
# t = preprocess_text(t)

In [105]:
# te = create_embedding(t, model, tokenizer)

In [195]:
if tokenizer2.pad_token is None:
        print('Noooooooo😱')

Noooooooo😱


In [197]:
# Add padding token if not available
if tokenizer2.pad_token is None:
        tokenizer2.add_special_tokens({'pad_token': '[PAD]'})

# Define the padding token id
pad_token_id = tokenizer2.pad_token_id

In [199]:
if tokenizer2.pad_token is None:
        print('Noooooooo😱')
else:
        print('Yes, you s..off.beep..')

Yes, you s..off.beep..


In [201]:
def main():

    # Directory containing PDF files
    pdf_directory = "."  # Replace with your actual path

    # List to store all blog data
    blogs = []

    # Process each PDF file
    for filename in tqdm(os.listdir(pdf_directory)):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, filename)
            
            # Extract and clean text
            text = extract_text_from_pdf(pdf_path)
            cleaned_text = clean_text(text)
            
            # Create embedding
            embedding = create_embedding(cleaned_text, model2, tokenizer2)
            
            # Store blog data
            blogs.append({
                "filename": filename,
                "text": cleaned_text,
                "embedding": embedding.tolist()  # Convert to list for JSON serialization
            })

    # Create FAISS index
    dimension = len(blogs[0]["embedding"][0])
    index = faiss.IndexFlatL2(dimension)
    
    # Add embeddings to the index
    embeddings = np.array([blog["embedding"][0] for blog in blogs])
    index.add(embeddings)

    # Save FAISS index
    faiss.write_index(index, "mental_health_blogs_dialgpt.index")

    # Save blog data (without embeddings) for later retrieval
    blog_data = [{k: v for k, v in blog.items() if k != "embedding"} for blog in blogs]
    with open("blog_data_dialgpt.json", "w") as f:
        json.dump(blog_data, f)

    print(f"Processed {len(blogs)} PDF files.")
    print("FAISS index saved as 'mental_health_blogs_dialgpt.index'")
    print("Blog data saved as 'blog_data_dialgpt.json'")

if __name__ == "__main__":
    main()

  0%|          | 0/15 [00:18<?, ?it/s]


AttributeError: 'CausalLMOutputWithCrossAttentions' object has no attribute 'last_hidden_state'

In [12]:
import json
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [122]:
index = faiss.read_index("mental_health_blogs.index")

with open("blog_data.json", "r") as f:
    data = json.load(f)

# Initialize the sentence transformer model
sentence_model = SentenceTransformer('mental/mental-bert-base-uncased', token=True)

No sentence-transformers model found with name mental/mental-bert-base-uncased. Creating a new one with mean pooling.
c:\Pyhton\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [155]:
def process_query(query, top_k=5):
    # Convert query to embedding
    query_embedding = create_embedding(query, model, tokenizer)[0]
    
    # Perform similarity search
    distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
    
    # Retrieve the most relevant articles
    relevant_articles = [data[i] for i in indices[0]]
    
    return relevant_articles

In [187]:
def generate_response(query, relevant_articles):
    # Prepare input text
    context = " ".join([article['text'] for article in relevant_articles])
    input_text = f"Query: {query}\nContext: {context}\nAnswer:"
    
    max_input_length = 1024
    if len(input_text) > max_input_length:
        input_text = input_text[:max_input_length]

    # Tokenize input
    inputs = tokenizer2(input_text, return_tensors="pt", max_length=1024, truncation=True)
    
    # Generate response
    with torch.no_grad():
        output = model2.generate(**inputs, max_length=200, num_beams=4, early_stopping=True)
    
    # Decode and return the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [188]:
def query_and_respond(query):
    relevant_articles = process_query(query)
    response = generate_response(query, relevant_articles)
    return response

In [189]:
user_query = "depression?"
result = query_and_respond(user_query)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [190]:
result

'##ptic [unused24] essex [unused29] [unused193] fairs [unused24] [unused350] [unused252] shipments [unused989] [unused384] [unused637] nikolai [unused321] [unused709] [unused302] 」 [unused529] investigations [unused261] [unused261] [unused388] [unused320] [unused340] ד や [unused302] [unused529] investigations նakes [unused335] [unused335] [unused333] ⁷ [unused725] [unused64] recently [unused538] ミ [unused340] ₚ 樹 [unused582] [unused510] [unused346] many testament post [unused507] martini [unused383] [unused282] [unused257] sights [unused281] [unused541] [unused698] [unused279] ニ [unused252] ধ never [unused281]ham [unused607] [unused455] [unused302] [unused252] rail awesome [unused314] stumble [unused976] [unused321] [unused313] [unused875] [unused285] [unused917] stumble [unused321] stone [unused314] byrne weather settings [unused895] [unused509] [unused505] [unused324] spoken [unused335] gene [unused498] 日 [unused279] five college besides [unused321] stone [unused314] generators conta

In [152]:
# a,b = index.search(create_embedding("depression?", model, tokenizer)[0].reshape(1,-1), 2)

In [161]:
# ra = [data[i] for i in b[0]]
c = " ".join([article['text'] for article in ra])

In [167]:
# ip = f"Query: 'depression?'\nContext: {c}\nAnswer:"
inputs = tokenizer(ip, return_tensors="pt", max_length=1024, truncation=True)

In [171]:
model.can_generate()

False

In [203]:
import os
import json
import numpy as np
import torch
import faiss
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, T5ForConditionalGeneration, T5Tokenizer

# Initialize the BERT model and tokenizer for embeddings
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
model = BertModel.from_pretrained(bert_model_name)


def extract_text_from_pdf(pdf_path):
    import PyPDF2
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def clean_text(text):
    import re
    url_pattern = re.compile(r'http\S+|www\S+')
    email_pattern = re.compile(r'\S+@\S+')
    date_time_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}')
    parentheses_pattern = re.compile(r'\(.*?\)')
    special_char_pattern = re.compile(r'[^\w\s\'"]')
    apostrophe_pattern = re.compile(r"’|‘")
    whitespace_pattern = re.compile(r'\s+')
    
    text = text.lower()
    text = url_pattern.sub('', text)
    text = email_pattern.sub('', text)
    text = date_time_pattern.sub('', text)
    text = parentheses_pattern.sub('', text)
    text = special_char_pattern.sub(' ', text)
    text = apostrophe_pattern.sub("'", text)
    text = whitespace_pattern.sub(' ', text)
    text = text.strip()
    text = ' '.join(text.split())
    return text

def create_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

def main():
    pdf_directory = "."  # Replace with your actual path
    blogs = []

    for filename in tqdm(os.listdir(pdf_directory)):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, filename)
            text = extract_text_from_pdf(pdf_path)
            cleaned_text = clean_text(text)
            embedding = create_embedding(cleaned_text, model, tokenizer)
            blogs.append({
                "filename": filename,
                "text": cleaned_text,
                "embedding": embedding.tolist()
            })

    dimension = len(blogs[0]["embedding"][0])
    index = faiss.IndexFlatL2(dimension)
    embeddings = np.array([blog["embedding"][0] for blog in blogs])
    index.add(embeddings)
    faiss.write_index(index, "mental_health_blogs_dialgpt.index")

    blog_data = [{k: v for k, v in blog.items() if k != "embedding"} for blog in blogs]
    with open("blog_data_dialgpt.json", "w") as f:
        json.dump(blog_data, f)

    print(f"Processed {len(blogs)} PDF files.")
    print("FAISS index saved as 'mental_health_blogs_dialgpt.index'")
    print("Blog data saved as 'blog_data_dialgpt.json'")


def process_query(query, top_k=5):
    query_embedding = create_embedding(query, model, tokenizer)[0]
    index = faiss.read_index("mental_health_blogs_dialgpt.index")
    
    distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
    with open("blog_data_dialgpt.json", "r") as f:
        data = json.load(f)
    relevant_articles = [data[i] for i in indices[0]]
    
    return relevant_articles
def generate_response(query, relevant_articles):
    context = " ".join([article['text'] for article in relevant_articles])
    input_text = f"Query: {query}\nContext: {context}\nAnswer:"
    
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=200, num_beams=4, early_stopping=True)
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

def query_and_respond(query):
    relevant_articles = process_query(query)
    response = generate_response(query, relevant_articles)
    return response

if __name__ == "__main__":
    main()
    user_query = "depression?"
    result = query_and_respond(user_query)
    print(result)


In [18]:
from pathlib import Path
# m = Path("C:\Users\Gourav\.cache\huggingface\hub\models--google--gemma-2b-it\blobs")
model_directory = "C:\\Users\\Gourav\\.cache\\huggingface\\hub\\models--google--gemma-2b-it"


In [19]:
# Load the tokenizer and model from this path
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = AutoModelForCausalLM.from_pretrained(model_directory)

OSError: C:\Users\Gourav\.cache\huggingface\hub\models--google--gemma-2b-it does not appear to have a file named config.json. Checkout 'https://huggingface.co/C:\Users\Gourav\.cache\huggingface\hub\models--google--gemma-2b-it/None' for available files.

In [7]:
save_directory = "./gemma_model"

# Create the directory if it does not exist
os.makedirs(save_directory, exist_ok=True)

# Save the tokenizer and model
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

NameError: name 'model' is not defined

In [ ]:
def main():

    # Directory containing PDF files
    pdf_directory = "."  # Replace with your actual path

    # List to store all blog data
    blogs = []

    # Process each PDF file
    for filename in tqdm(os.listdir(pdf_directory)):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, filename)
            
            # Extract and clean text
            text = extract_text_from_pdf(pdf_path)
            cleaned_text = clean_text(text)
            
            # Create embedding
            embedding = create_embedding(cleaned_text, model2, tokenizer2)
            
            # Store blog data
            blogs.append({
                "filename": filename,
                "text": cleaned_text,
                "embedding": embedding.tolist()  # Convert to list for JSON serialization
            })

    # Create FAISS index
    dimension = len(blogs[0]["embedding"][0])
    index = faiss.IndexFlatL2(dimension)
    
    # Add embeddings to the index
    embeddings = np.array([blog["embedding"][0] for blog in blogs])
    index.add(embeddings)

    # Save FAISS index
    faiss.write_index(index, "mental_health_blogs_dialgpt.index")

    # Save blog data (without embeddings) for later retrieval
    blog_data = [{k: v for k, v in blog.items() if k != "embedding"} for blog in blogs]
    with open("blog_data_dialgpt.json", "w") as f:
        json.dump(blog_data, f)

    print(f"Processed {len(blogs)} PDF files.")
    print("FAISS index saved as 'mental_health_blogs_dialgpt.index'")
    print("Blog data saved as 'blog_data_dialgpt.json'")

if __name__ == "__main__":
    main()

c:\Pyhton\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [2]:
def process_query(query, index, top_k=5):
    query_embedding = create_embedding(query, embedding_model, embedding_tokenizer)[0]
    distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
    return indices[0]

def generate_response(query, context):
    input_text = f"Query: {query}\nContext: {context}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    
    with torch.no_grad():
        output = model.generate(**inputs, max_length=200, num_beams=4, early_stopping=True)
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

def query_and_respond(query, index, blog_data):
    relevant_indices = process_query(query, index)
    context = " ".join([blog_data[i]['text'] for i in relevant_indices])
    response = generate_response(query, context)
    return response

In [3]:
if __name__ == "__main__":
    main()

    # Load the saved index and blog data
    index = faiss.read_index("mental_distil.index")
    with open("blog_distil.json", "r") as f:
        blog_data = json.load(f)

    # Example usage
    user_query = "What are some strategies for managing depression?"
    result = query_and_respond(user_query, index, blog_data)
    print(result)

100%|██████████| 20/20 [00:11<00:00,  1.70it/s]


Processed 11 PDF files.
FAISS index saved as 'mental_health_blogs.index'
Blog data saved as 'blog_data.json'


: 